#  Submit geo opt for molecules, slabs and bulks

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# General imports.
import ipywidgets as ipw
from IPython.display import clear_output

# AiiDA imports.
%load_ext aiida
%aiida
from aiida import orm, plugins

# AiiDAlab imports.
import aiidalab_widgets_base as awb

# Custom imports.
from surfaces_tools.widgets.build_slab import BuildSlab
from surfaces_tools.widgets.computational_resources import ProcessResourcesWidget, ResourcesEstimatorWidget
from surfaces_tools.widgets.inputs import InputDetails
from surfaces_tools.widgets.empa_viewer import EmpaStructureViewer
from surfaces_tools.utils import wfn
from surfaces_tools.widgets.analyze_structure import StructureAnalyzer
from surfaces_tools.widgets.cdxml import CdxmlUpload2GnrWidget

In [ ]:
Cp2kGeoOptWorkChain = plugins.WorkflowFactory('nanotech_empa.cp2k.geo_opt')

In [ ]:
# Structure selector.
empa_viewer = EmpaStructureViewer()
build_slab = BuildSlab(title='Build slab')
ipw.dlink((empa_viewer, 'details'), (build_slab, 'details'))
ipw.dlink((empa_viewer, 'structure'), (build_slab, 'molecule'))

structure_selector = awb.StructureManagerWidget(
    viewer=empa_viewer,
    importers=[
        awb.StructureUploadWidget(title="Import from computer"),
        awb.StructureBrowserWidget(title="AiiDA database"),
        awb.OptimadeQueryWidget(embedded=True),
        awb.SmilesWidget(title="From SMILES"),
        CdxmlUpload2GnrWidget(title="CDXML"),
    ],
    editors = [
        awb.BasicStructureEditor(title="Edit structure"),
        build_slab
    ],
    storable=False, node_class='StructureData')
display(structure_selector)

# Code.
computational_resources = awb.ComputationalResourcesWidget(
            description="CP2K code:", default_calc_job_plugin="cp2k"
        )
resources = ProcessResourcesWidget()

input_details = InputDetails()
create_input = ipw.Button(description="Create Input")

# Resources

In [ ]:
ipw.dlink((empa_viewer, 'details'), (input_details, 'details'))
ipw.dlink((computational_resources, 'value'),(input_details, 'selected_code'))

def prepare_geo_opt():
    parameters = input_details.final_dictionary.copy()

    builder = Cp2kGeoOptWorkChain.get_builder()
    if 'cell_opt' in parameters['dft_params']:
        builder.metadata.label += "CP2K_CellOpt"
    else:
        builder.metadata.label = "CP2K_GeoOpt"
    builder.metadata.description = parameters['description']
    code = orm.load_code(computational_resources.value)
    builder.code = code
    builder.options = {
            "max_wallclock_seconds": resources.walltime_seconds,
            "resources": {
                'num_machines': resources.nodes,
                'num_mpiprocs_per_machine': resources.tasks_per_node,
                'num_cores_per_mpiproc': resources.threads_per_task,
            },
        }
    
    builder.structure = structure_selector.structure_node
    
    # Check if a restart wfn is available.
    wave_function = None
    if structure_selector.structure_node.is_stored:
        wave_function = wfn.structure_available_wfn(
        node=structure_selector.structure_node,
        relative_replica_id=None,
        current_hostname=code.computer.hostname,
        return_path=False,
        dft_params=parameters['dft_params'],
        )
    if wave_function is not None:
        print(f"Restarting from wfn in folder: {wave_function.pk}")
        builder.parent_calc_folder = wave_function
    

    builder.dft_params = Dict(parameters['dft_params'])
    builder.sys_params = Dict(parameters['sys_params'])
   
    return builder

In [ ]:
def after_submission(_=None):   
    structure_selector.value = None

btn_submit = awb.SubmitButtonWidget(Cp2kGeoOptWorkChain, inputs_generator=prepare_geo_opt)
btn_submit.on_submitted(after_submission)

In [ ]:
output = ipw.Output()
def update_all(_=None):
    btn_submit.btn_submit.disabled=True 

def create(_=None):
    if computational_resources.value is None:
        msg = 'Please select a code first.'
        can_submit = False
    else:            
        can_submit, msg, details_dict = input_details.return_final_dictionary()
 
    with output:
        clear_output()
        if can_submit:
            print("Ready to submit a ",details_dict['system_type']," optimization" )
            btn_submit.btn_submit.disabled = not can_submit
            display(btn_submit)
        else:
            print(msg)

create_input.on_click(create)
input_details.observe(update_all, names='details')

In [ ]:
# Resources estimation.
resources_estimation = ResourcesEstimatorWidget()
resources_estimation.link_to_resources_widget(resources)
ipw.dlink((structure_selector, 'structure'), (input_details, 'ase_atoms'))
ipw.dlink((empa_viewer, 'details'), (resources_estimation, 'details'))
ipw.dlink((input_details, 'uks'), (resources_estimation, 'uks'))
_ = ipw.dlink((computational_resources, 'value'), (resources_estimation, 'selected_code'))

In [ ]:
display(ipw.VBox([input_details, resources, computational_resources,resources_estimation,  create_input]), output)